In [1]:
import pandas as pd
import datetime

In [2]:
# today's data and yesterday's date
today = (datetime.datetime.today().strftime('%d%b%Y')).upper()
yesterday = ((datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%d%b%Y')).upper()
today, yesterday

('20OCT2022', '19OCT2022')

In [3]:
# source url for fetching the data from nseIndia.com
source_url = 'https://archives.nseindia.com/content/historical/DERIVATIVES/2022/OCT/fo{}bhav.csv.zip'.format(today)  # CHANGE-ME
source_url

'https://archives.nseindia.com/content/historical/DERIVATIVES/2022/OCT/fo20OCT2022bhav.csv.zip'

In [4]:
# fetching the zip file
# fetching the csv file 
# NOTE : the zipfile must contain only csv file nothing else
temp_data = pd.read_csv(source_url,compression='zip')

In [5]:
# Dropping the unnecessary columns
temp_data = temp_data.drop(columns=['STRIKE_PR', 'OPEN', 'OPTION_TYP', 'HIGH', 'LOW', 'TIMESTAMP'])

In [6]:
# Dropping the last column as it was empty
temp_data.drop(temp_data.columns[[9]], axis=1, inplace=True)

In [7]:
# NIFTY 100 STOCK NAMES
companies = """ADANITRANS
HAL
BEL
SBIN
BANKBARODA
ADANIPORTS
SBICARD
EICHERMOT
SIEMENS
ITC
SBILIFE
NESTLEIND
ADANIENT
CHOLAFIN
BHARTIARTL
INDUSINDBK
INDIGO
HAVELLS
TATAMOTORS
DLF
M&M
APOLLOHOSP
LT
RELIANCE
NAUKRI
CIPLA
LTI
MUTHOOTFIN
HDFCLIFE
COALINDIA
HCLTECH
HINDALCO
TATAPOWER
ATGL
ICICIGI
TATACONSUM
ICICIBANK
BIOCON
ULTRACEMCO
TATASTEEL
WIPRO
HDFCAMC
IOC
MCDOWELL-N
ASIANPAINT
BAJAJHLDNG
PIIND
BERGEPAINT
INDUSTOWER
INFY
JSWSTEEL
MARUTI
SHREECEM
TCS
ONGC
ZOMATO
BOSCHLTD
TITAN
ICICIPRULI
BANDHANBNK
COLPAL
PIDILITIND
HINDUNILVR
UPL
POWERGRID
SRF
BAJFINANCE
MARICO
MPHASIS
IRCTC
BAJAJFINSV
ADANIGREEN
AXISBANK
HEROMOTOCO
DRREDDY
DABUR
GODREJCP
VEDL
PGHH
DIVISLAB
KOTAKBANK
GLAND
GAIL
LICI
BAJAJ-AUTO
BPCL
GRASIM
HDFCBANK
TECHM
DMART
BRITANNIA
SUNPHARMA
HDFC
NTPC
TORNTPHARM
NYKAA
PAYTM
AMBUJACEM
ACC
MOTHERSON
BANKNIFTY
NIFTY
"""
companies = companies.split('\n')
len(companies)

103

In [8]:
# Inplace droppping the row with symbol as FINNIFTY
# temp_data.drop(temp_data[temp_data['SYMBOL'] == 'FINNIFTY'].index, inplace=True)

# Filtering out data based on INSTRUMENT AND SYMBOL
temp_data = temp_data.loc[temp_data['INSTRUMENT'].isin(['FUTSTK', 'FUTIDX'])]
temp_data = temp_data.loc[temp_data['SYMBOL'].isin(companies)]

In [9]:
# Filtering out data based on current month expiry date
temp_data = temp_data[temp_data['EXPIRY_DT'].astype(str).str.match(r"\d{2}-Oct-2022")]

In [10]:
# Dropping out the first column i.e. INSTRUMENT
temp_data.drop(temp_data.columns[[0]], axis=1, inplace=True)

In [11]:
# new Column names (added date with the names)
column_names = [ 'SYMBOL', 'EXPIRY_DT', 'CLOSE'+today, 'SETTLE_PR'+today, 'CONTRACTS'+today, 'VAL_INLAKH'+today,'OPEN_INT'+today, 'CHG_IN_OI'+today]

# assigning the new column names
temp_data.columns = column_names

In [12]:
temp_data

,SYMBOL,EXPIRY_DT,CLOSE20OCT2022,SETTLE_PR20OCT2022,CONTRACTS20OCT2022,VAL_INLAKH20OCT2022,OPEN_INT20OCT2022,CHG_IN_OI20OCT2022
0,BANKNIFTY,27-Oct-2022,40205.35,40205.35,160299,1605768.35,1519525,-68225
16,NIFTY,27-Oct-2022,17540.70,17540.70,189528,1655734.25,11338400,351250
34,ACC,27-Oct-2022,2286.10,2286.10,6140,34795.23,3218750,-478250
37,ADANIENT,27-Oct-2022,3358.10,3358.10,14473,240114.19,8942500,-3338500
40,ADANIPORTS,27-Oct-2022,819.70,819.70,8830,89468.41,29191250,-3873750
...,...,...,...,...,...,...,...,...
561,TORNTPHARM,27-Oct-2022,1593.60,1593.60,1150,9146.05,1416000,-246000
576,ULTRACEMCO,27-Oct-2022,6317.60,6317.60,9621,61010.07,1675000,-335000
579,UPL,27-Oct-2022,712.85,712.85,13879,125267.37,17407000,-4693000
582,VEDL,27-Oct-2022,283.50,283.50,8088,35040.25,23307350,-4980150


In [13]:
# writing the today's data to temp file to refresh the index.
temp_data.to_csv('temp_data.csv', index=False)

In [14]:
# reading the today's data from the temp file.
today_data = pd.read_csv('temp_data.csv')

# reading the main data from the mainfile
main_data = pd.read_csv('fandodata.csv')


In [15]:
# removing today's data's  expiry column as not required during merge
today_data.drop(columns='EXPIRY_DT', inplace=True)

In [16]:
# merging today's data with main data
main_data = main_data.merge(today_data, how='outer', on='SYMBOL')

In [17]:
# rearrange data
header_main = ['CLOSE'+yesterday, 'SETTLE_PR'+yesterday, 'CONTRACTS'+yesterday, 'VAL_INLAKH'+yesterday,'OPEN_INT'+yesterday, 'CHG_IN_OI'+yesterday]
header_new = ['CLOSE'+today, 'SETTLE_PR'+today, 'CONTRACTS'+today, 'VAL_INLAKH'+today,'OPEN_INT'+today, 'CHG_IN_OI'+today]

for i in range(6):
    new_head = header_new[i]
    prev_head = main_data.columns.get_loc(header_main[i]) + 1

    extracted_col = main_data[new_head]
    main_data.drop(columns=new_head, inplace=True)
    main_data.insert(prev_head, new_head, extracted_col)
    

In [18]:
main_data

,SYMBOL,EXPIRY_DT,CLOSE18OCT2022,CLOSE19OCT2022,CLOSE20OCT2022,SETTLE_PR18OCT2022,SETTLE_PR19OCT2022,SETTLE_PR20OCT2022,CONTRACTS18OCT2022,CONTRACTS19OCT2022,CONTRACTS20OCT2022,VAL_INLAKH18OCT2022,VAL_INLAKH19OCT2022,VAL_INLAKH20OCT2022,OPEN_INT18OCT2022,OPEN_INT19OCT2022,OPEN_INT20OCT2022,CHG_IN_OI18OCT2022,CHG_IN_OI19OCT2022,CHG_IN_OI20OCT2022
0,BANKNIFTY,27-Oct-22,40380.10,40401.75,40205.35,40380.10,40401.75,40205.35,124441,150366,160299,1255775.74,1521117.53,1605768.35,1633100,1587750,1519525,-194875,-45350,-68225
1,NIFTY,27-Oct-22,17502.00,17494.65,17540.70,17502.00,17494.65,17540.70,190681,170212,189528,1667618.58,1492063.86,1655734.25,11411250,10987150,11338400,-1273300,-424100,351250
2,ACC,27-Oct-22,2224.85,2247.75,2286.10,2224.85,2247.75,2286.10,10937,6089,6140,61276.54,34193.27,34795.23,4131250,3697000,3218750,-530500,-434250,-478250
3,ADANIENT,27-Oct-22,3291.45,3275.35,3358.10,3291.45,3275.35,3358.10,8850,7307,14473,145049.84,120271.07,240114.19,13333500,12281000,8942500,-1091000,-1052500,-3338500
4,ADANIPORTS,27-Oct-22,810.00,805.25,819.70,810.00,805.25,819.70,36783,9267,8830,372577.41,94238.16,89468.41,35147500,33065000,29191250,-34007500,-2082500,-3873750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,TORNTPHARM,27-Oct-22,1564.55,1580.40,1593.60,1564.55,1580.40,1593.60,906,777,1150,7112.34,6117.27,9146.05,1736500,1662000,1416000,18500,-74500,-246000
87,ULTRACEMCO,27-Oct-22,6368.65,6415.05,6317.60,6368.65,6415.05,6317.60,11133,23359,9621,71131.67,149183.53,61010.07,1871700,2010000,1675000,-40000,138300,-335000
88,UPL,27-Oct-22,683.75,678.30,712.85,683.75,678.30,712.85,1808,2472,13879,16101.66,21879.00,125267.37,23280400,22100000,17407000,-465400,-1180400,-4693000
89,VEDL,27-Oct-22,280.60,277.30,283.50,280.60,277.30,283.50,4421,4957,8088,19324.79,21506.42,35040.25,29367850,28287500,23307350,-94550,-1080350,-4980150


In [19]:
main_data.to_csv('fandodata.csv', index=False)